# SQL Refresher 1 c

## imports

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

## my_select_query_pandas() - function to run a select query and return rows in a Pandas dataframe

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

## Connect to the Postgres database

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

## Create a cursor for the connection

In [4]:
cursor = connection.cursor()

# Lab: SQL - INSERT Statements

## Create a simple temporary table to demonstrate transaction processing concepts; note that where 0 = 1 is a simple trick to create an empty table

In [5]:
connection.rollback()

query = """

drop table if exists temp_test_insert
;

create table temp_test_insert 
as 
select zip, city, state
from zip_codes
where 0 = 1;

"""

cursor.execute(query)

connection.commit()

## Query the table temp_test_insert from SQL_Refresher_2

## Insert the rows returned from a query into a table

In [6]:
# NOTE: this time we are not going to rollback nor commit
#       inserts will only be visible in this transaction
#       until we commit

query = """

insert into temp_test_insert
select zip, city, state
from zip_codes
where city = 'Berkeley' and state = 'CA'

"""

cursor.execute(query)


## At this point, rows are visible to us, but  not visible to other sessions, even from the same user!

## commit - saves our changes, ends our current transaction, creates a new transaction, changes are now visible to other sessions

## rollback - discards our changes, ends our current transaciton, creates a new transaction

In [7]:
# NOTE: set the rollback flags to False
#       for this example

rollback_before_flag = False
rollback_after_flag = False

query = """

select * 
from temp_test_insert

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state
0,94702,Berkeley,CA
1,94703,Berkeley,CA
2,94704,Berkeley,CA
3,94705,Berkeley,CA
4,94707,Berkeley,CA
5,94708,Berkeley,CA
6,94709,Berkeley,CA
7,94710,Berkeley,CA
8,94720,Berkeley,CA


## Query the table temp_test_insert from SQL_Refresher_2

## commit our changes

In [8]:
# NOTE: once we commit, other transactions can see our changes

connection.commit()

## Query the table temp_test_insert from SQL_Refresher_2

## Individual insert statements not from a query; if we give a column list, we have to provide data for all columns in the right order; we can give a column list 

In [9]:
# NOTE: this time we are not going to rollback nor commit
#       inserts will only be visible in this transaction
#       until we commit

query = """

insert into temp_test_insert
values 
('90001', 'Los Angeles', 'CA');

insert into temp_test_insert
(zip, city, state)
values 
('90002', 'Los Angeles', 'CA');

insert into temp_test_insert
(city, state, zip)
values
('Los Angeles', 'CA', '90003');

insert into temp_test_insert
(zip)
values
('90004');

insert into temp_test_insert
(zip)
values
('98101');

insert into temp_test_insert
(zip)
values
('98102');

insert into temp_test_insert
(zip)
values
('98103');

insert into temp_test_insert
(zip)
values
('98104');


"""

cursor.execute(query)


## Verify the data in the table

In [10]:
# NOTE: set the rollback flags to False
#       for this example

rollback_before_flag = False
rollback_after_flag = False

query = """

select * 
from temp_test_insert

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state
0,94702,Berkeley,CA
1,94703,Berkeley,CA
2,94704,Berkeley,CA
3,94705,Berkeley,CA
4,94707,Berkeley,CA
5,94708,Berkeley,CA
6,94709,Berkeley,CA
7,94710,Berkeley,CA
8,94720,Berkeley,CA
9,90001,Los Angeles,CA


## Query the table temp_test_insert from SQL_Refresher_2

## commit our changes

In [11]:
# NOTE: once we commit, other transactions can see our changes

connection.commit()

## Query the table temp_test_insert from SQL_Refresher_2

# Lab: SQL - UPDATE Statements

## update a single row in a table

In [12]:
# NOTE: this time we are not going to rollback nor commit
#       inserts will only be visible in this transaction
#       until we commit

query = """

update temp_test_insert
set city = 'Seattle', state = 'WA'
where zip = '98104'
"""

cursor.execute(query)


## Verify our update

In [13]:
# NOTE: set the rollback flags to False
#       for this example

rollback_before_flag = False
rollback_after_flag = False

query = """

select * 
from temp_test_insert

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state
0,94702,Berkeley,CA
1,94703,Berkeley,CA
2,94704,Berkeley,CA
3,94705,Berkeley,CA
4,94707,Berkeley,CA
5,94708,Berkeley,CA
6,94709,Berkeley,CA
7,94710,Berkeley,CA
8,94720,Berkeley,CA
9,90001,Los Angeles,CA


## Query the table temp_test_insert from SQL_Refresher_2

## update multiple rows in a table with data from another table

In [14]:
# NOTE: this time we are not going to rollback nor commit
#       inserts will only be visible in this transaction
#       until we commit

query = """

update temp_test_insert as t
set city = z.city,
    state = z.state
from zip_codes as z
where z.zip = t.zip

"""

cursor.execute(query)


## Query the table temp_test_insert from SQL_Refresher_2

## Verify data from our last updates

In [15]:
# NOTE: set the rollback flags to False
#       for this example

rollback_before_flag = False
rollback_after_flag = False

query = """

select * 
from temp_test_insert

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state
0,94702,Berkeley,CA
1,94703,Berkeley,CA
2,94704,Berkeley,CA
3,94705,Berkeley,CA
4,94707,Berkeley,CA
5,94708,Berkeley,CA
6,94709,Berkeley,CA
7,94710,Berkeley,CA
8,94720,Berkeley,CA
9,90001,Los Angeles,CA


## Query the table temp_test_insert from SQL_Refresher_2

## commit our changes

In [16]:
# NOTE: once we commit, other transactions can see our changes

connection.commit()

## Query the table temp_test_insert from SQL_Refresher_2

# Lab: SQL - DELETE Statements

## delete a single row from a table

In [17]:
# NOTE: this time we are not going to rollback nor commit
#       inserts will only be visible in this transaction
#       until we commit

query = """

delete 
from temp_test_insert
where zip = '94702'


"""

cursor.execute(query)


## Verify the delete

In [18]:
# NOTE: set the rollback flags to False
#       for this example

rollback_before_flag = False
rollback_after_flag = False

query = """

select * 
from temp_test_insert

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state
0,94703,Berkeley,CA
1,94704,Berkeley,CA
2,94705,Berkeley,CA
3,94707,Berkeley,CA
4,94708,Berkeley,CA
5,94709,Berkeley,CA
6,94710,Berkeley,CA
7,94720,Berkeley,CA
8,90001,Los Angeles,CA
9,90002,Los Angeles,CA


## delete multiple rows from a table

In [19]:
# NOTE: this time we are not going to rollback nor commit
#       inserts will only be visible in this transaction
#       until we commit

query = """

delete 
from temp_test_insert
where city = 'Seattle';


"""

cursor.execute(query)


## Verify our deletes

In [20]:
# NOTE: set the rollback flags to False
#       for this example

rollback_before_flag = False
rollback_after_flag = False

query = """

select * 
from temp_test_insert

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state
0,94703,Berkeley,CA
1,94704,Berkeley,CA
2,94705,Berkeley,CA
3,94707,Berkeley,CA
4,94708,Berkeley,CA
5,94709,Berkeley,CA
6,94710,Berkeley,CA
7,94720,Berkeley,CA
8,90001,Los Angeles,CA
9,90002,Los Angeles,CA


## rollback our changes

In [21]:
connection.rollback()

## Verify the rollback of our changes

In [22]:
# NOTE: set the rollback flags to False
#       for this example

rollback_before_flag = False
rollback_after_flag = False

query = """

select * 
from temp_test_insert

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state
0,94702,Berkeley,CA
1,94703,Berkeley,CA
2,94704,Berkeley,CA
3,94705,Berkeley,CA
4,94707,Berkeley,CA
5,94708,Berkeley,CA
6,94709,Berkeley,CA
7,94710,Berkeley,CA
8,94720,Berkeley,CA
9,90001,Los Angeles,CA


## delete all the rows from the table; dangerous!; if you make a mistake and leave off a where clause in a delete, all rows are deleted!

In [23]:
# NOTE: this time we are not going to rollback nor commit
#       inserts will only be visible in this transaction
#       until we commit

query = """

delete 
from temp_test_insert;


"""

cursor.execute(query)


## Verify all rows deleted from the table

In [24]:
# NOTE: set the rollback flags to False
#       for this example

rollback_before_flag = False
rollback_after_flag = False

query = """

select * 
from temp_test_insert

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state


## Query the table temp_test_insert from SQL_Refresher_2

## commit our deletes

In [25]:
# NOTE: once we commit, other transactions can see our changes

connection.commit()

## Query the table temp_test_insert from SQL_Refresher_2

## Drop our table

In [26]:
connection.rollback()

query = """

drop table if exists temp_test_insert
;

"""

cursor.execute(query)

connection.commit()